In [4]:
from pathlib import Path
import json
from more_itertools import with_iter
import pandas as pd
import numpy as np
from functools import partial
import cv2
import pytesseract
import matplotlib.pyplot as plt
import re
import difflib

In [5]:
with open('fighters.json','r') as fh:
    list_of_fighters = lof = list(map(str.upper, json.load(fh)))

In [6]:
annotations_path = Path('data/annotations')
afields = []

In [7]:
s1ar = []
for p in annotations_path.glob('*STATS-1.json'):
    with open(p) as po:
        pot = json.load(po)
        s1ar.append(pot)
s1ar = [[{k:v for k,v in s.items() if k in {'points','label'}} for s in a['shapes']] for a in s1ar]

In [8]:
s = []
for l in s1ar:
    for k in l:
        s.append(k)
s = pd.DataFrame(s)

In [9]:
x = (s[s['label']=='1 right name'])['points'].map(lambda l: cv2.boundingRect(np.array(l)))
x,y, dx, dy = list(map(int,np.mean(np.array(list(x.map(list)),dtype='int'), axis=0)))
x, y, dx, dy

(348, 330, 204, 74)

In [18]:
image_set = Path('images').glob('*-STATS-1.png')

0 = Orientation and script detection (OSD) only.
1 = Automatic page segmentation with OSD.
2 = Automatic page segmentation, but no OSD, or OCR. (not implemented)
3 = Fully automatic page segmentation, but no OSD. (Default)
4 = Assume a single column of text of variable sizes.
5 = Assume a single uniform block of vertically aligned text.
6 = Assume a single uniform block of text.
7 = Treat the image as a single text line.
8 = Treat the image as a single word.
9 = Treat the image as a single word in a circle.
10 = Treat the image as a single character.
11 = Sparse text. Find as much text as possible in no particular order.
12 = Sparse text with OSD.
13 = Raw line. Treat the image as a single text line,
     bypassing hacks that are Tesseract-specifi
     
4 seems best for now...
Struct elem (2,3)

In [61]:
def printable(s):
    ts = re.sub(r'[^\w\s]','',s, re.UNICODE)
    return re.sub(r'_','',ts).strip()

In [106]:
def process_image(roi):
    #roi = cv2.GaussianBlur(roi,(3,3),0)
    rv, i = cv2.threshold(roi, 180, 255, cv2.THRESH_BINARY_INV)
    #i = cv2.GaussianBlur(roi,(3,3),0)
    # Doesn't work as well
    #roi=cv2.cvtColor(roi, cv2.COLOR_RGB2GRAY)
    #i = cv2.adaptiveThreshold(roi,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,55,0)
    kernel = cv2.getStructuringElement(cv2.MORPH_CROSS,(2,3))
    erosion = cv2.dilate(i,kernel,iterations = 1)
    return erosion

def ocr_image(erosion):
    #config = (r'--psm 4',  r'--oem 1', r'--user-patterns "/home/nate/Projects/clb4s/fightingwords.user-patterns"', r'--lang "eng"', r'--tessdata-dir "/home/nate/Projects/clb4s/tessdata/"', r'"bazaar"')
    #config = r'blah blah bl;ah --psm 5 --oem 1 --tessdata-dir "/usr/share/tesseract-ocr/4.00/tessdata" -c user-patterns="/home/nate/Projects/clb4s/fightingworrrds.user-patterns" "bazaar"'
    config = r'--psm 4 --oem 1 --user-patterns "/home/nate/Projects/clb4s/fightingwords.user-patterns"'
    tff = pytesseract.image_to_string(erosion, config=config)
    return tff

def extract_roi(image,x,y,dx,dy, m=(10,10)):
    # m is (y,x)
    roi = image[y+m[0]:y+dy-m[0],x+m[1]:x+dx-m[1]]
    return roi

def process_text(s):
    s = s.replace('\n',' ').upper()
    s = printable(s)
    candidates = difflib.get_close_matches(s, lof, n=1, cutoff=0.6)
    return candidates[0] if len(candidates)>0 else None

In [116]:
atf = list()
bail_after = 69
k = 0
for ip in image_set:
    if k > bail_after:
        break
    k += 1
    image = cv2.imread(str(ip))

    roi = extract_roi(image,x,y,dx,dy,(10,10))
    roi = process_image(roi)
    tff = ocr_image(roi)
    fighter = process_text(tff)
    if fighter:
        atf.append(fighter)

In [119]:
# NORMAL
#atf = list(map(lambda s: s.replace('\n',' ').upper(), atf))
#atf = list(map(printable, atf))
#list(map(lambda s: (s,difflib.get_close_matches(s, lof, n=1, cutoff=0.6)), atf))
len(list(set(atf)))

47

In [118]:
atf

['NESS',
 'LUCAS',
 'YOUNG LINK',
 'PIKACHU',
 'LITTLE MAC',
 'RYU',
 'INCINEROAR',
 'DR. MARIO',
 'DIDDY KONG',
 'WII FIT TRAINER',
 'PIKACHU',
 'GANONDORF',
 'MEGA MAN',
 'CAPTAIN FALCON',
 'NESS',
 'LUCAS',
 'GANONDORF',
 'WARIO',
 'LINK',
 'ISABELLE',
 'PIT',
 'DARK SAMUS',
 'DARK SAMUS',
 'DUCK HUNT',
 'LINK',
 'PAC-MAN',
 'LUCAS',
 'META KNIGHT',
 'ROSALINA & LUMA',
 'SNAKE',
 'CHROM',
 'RICHTER',
 'LUCARIO',
 'SIMON',
 'PIKACHU',
 'DIDDY KONG',
 'LINK',
 'SHEIK',
 'SNAKE',
 'LINK',
 'TOON LINK',
 'CLOUD',
 'LUCARIO',
 'PEACH',
 'YOSHI',
 'ZERO SUIT SAMUS',
 'LUCARIO',
 'MARIO',
 'DARK PIT',
 'BOWSER JR.',
 'PALUTENA',
 'SAMUS',
 'LUCINA',
 'FOX',
 'DONKEY KONG',
 'MEGA MAN',
 'ZELDA',
 'CORRIN',
 'YOUNG LINK',
 'CHROM',
 'KING K. ROOL',
 'JIGGLYPUFF',
 'ZELDA',
 'WARIO',
 'LUCAS',
 'ROY',
 'RYU',
 'DARK PIT',
 'VILLAGER',
 'JIGGLYPUFF']

In [120]:
afields

[]

In [121]:
s1ar

[[{'label': '1 left place',
   'points': [[216, 2], [212, 110], [296, 110], [297, 0]]},
  {'label': '1 left name',
   'points': [[87, 356], [85, 406], [289, 380], [288, 331]]},
  {'label': '1 right place',
   'points': [[451, 0], [450, 127], [552, 124], [554, 0]]},
  {'label': '1 right name',
   'points': [[347, 355], [349, 402], [550, 379], [553, 329]]}],
 [{'label': '1 left place',
   'points': [[197, 3], [199, 108], [294, 109], [294, 0]]},
  {'label': '1 left name',
   'points': [[84, 356], [85, 404], [289, 376], [289, 329]]},
  {'label': '1 right place',
   'points': [[476, 3], [477, 118], [567, 109], [564, 0]]},
  {'label': '1 right name',
   'points': [[348, 356], [350, 405], [552, 374], [550, 329]]}],
 [{'label': '1 left place',
   'points': [[214, 1], [214, 119], [298, 117], [296, 0]]},
  {'label': '1 left name',
   'points': [[87, 358], [87, 404], [287, 376], [288, 330]]},
  {'label': '1 right place',
   'points': [[463, 1], [459, 120], [559, 120], [559, 0]]},
  {'label': '1 r